In [9]:
VERSION = 4.0

In [10]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.14.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
import pandas as pd
import numpy as np
from pyswarm import pso
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import mean_squared_error

In [12]:
data = pd.read_csv('training.csv', header=None)
data = data * (10**3)
data.head()

,0,1,2,3,4
0,-145.0,-145.0,-145.0,-145.0,-145.0
1,-145.0,-145.0,-145.0,-145.0,-145.0
2,-145.0,-145.0,-135.0,-145.0,-120.0
3,-135.0,-145.0,-160.0,-155.0,-150.0
4,-160.0,-155.0,-175.0,-180.0,-160.0


In [13]:
# Membuat dataset TensorFlow dari data Anda
# Menggunakan data yang telah Anda baca dari berkas CSV
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

print("x", X)
print("y", y)
print("min", X.min(), "max", X.max())

x [[-145. -145. -145. -145.]
 [-145. -145. -145. -145.]
 [-145. -145. -135. -145.]
 ...
 [-210. -210. -205. -210.]
 [-205. -210. -225. -220.]
 [-225. -220. -210. -205.]]
y [-145. -145. -120. ... -205. -215. -235.]
min -3095.0 max 2495.0


In [14]:
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batch_size)

# Menambahkan metode prefetch() ke dataset
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [15]:
tf.debugging.set_log_device_placement(True)
index = 0

# Define global variables for the number of weights and biases
num_input_units = data.shape[1] - 1
num_layers_units = [20, 30, 20]
num_output_units = 1

num_weights_layer1 = num_input_units * num_layers_units[0]
num_bias_layer1 = num_layers_units[0]
num_weights_layer2 = num_layers_units[0] * num_layers_units[1]
num_bias_layer2 = num_layers_units[1]
num_weights_layer3 = num_layers_units[1] * num_layers_units[2]
num_bias_layer3 = num_layers_units[2]
num_weights_layer4 = num_layers_units[2] * num_output_units
num_bias_layer4 = num_output_units

# Print the values
print("num_weights_layer1:", num_weights_layer1)
print("num_bias_layer1:", num_bias_layer1)
print("num_weights_layer2:", num_weights_layer2)
print("num_bias_layer2:", num_bias_layer2)
print("num_weights_layer3:", num_weights_layer3)
print("num_bias_layer3:", num_bias_layer3)
print("num_weights_layer4:", num_weights_layer4)
print("num_bias_layer4:", num_bias_layer4)

# Calculate the number of parameters for each layer
num_params_layer1 = (num_input_units + 1) * \
    num_layers_units[0]  # +1 for biases
num_params_layer2 = (num_layers_units[0] + 1) * num_layers_units[1]
num_params_layer3 = (num_layers_units[1] + 1) * num_layers_units[2]
num_params_layer4 = (num_layers_units[2] + 1) * num_output_units

# Calculate the total number of parameters
total_parameters = num_params_layer1 + num_params_layer2 + \
    num_params_layer3 + num_params_layer4


print(num_input_units, num_layers_units, num_output_units, num_weights_layer1, total_parameters)

# Define the fitness function to be minimized (MSE)
def fitness_function(params):
    global index
    print(f'Proses ke-{index}')

    # Create the ANN model with the specified architecture
    model = keras.Sequential()
    model.add(keras.layers.Dense(num_layers_units[0], input_dim=num_input_units, activation='sigmoid'))
    model.add(keras.layers.Dense(num_layers_units[1], activation='sigmoid'))
    model.add(keras.layers.Dense(num_layers_units[2], activation='sigmoid'))
    model.add(keras.layers.Dense(num_output_units, activation='sigmoid'))
    
    # Set the weights and biases of the model based on the PSO parameters
    print("Params", params[-num_weights_layer4 -
          num_bias_layer4:-num_bias_layer4].shape, num_weights_layer1)
    weights_layer1 = params[:num_weights_layer1].reshape((num_input_units, num_layers_units[0]))
    bias_layer1 = params[num_weights_layer1:num_weights_layer1 + num_bias_layer1]
    weights_layer2 = params[num_weights_layer1 + num_bias_layer1 : num_weights_layer1 + num_bias_layer1 + num_weights_layer2].reshape((num_layers_units[0], num_layers_units[1]))
    bias_layer2 = params[num_weights_layer1 + num_bias_layer1 + num_weights_layer2 : num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2]
    weights_layer3 = params[num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2 : num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2 + num_weights_layer3].reshape((num_layers_units[1], num_layers_units[2]))
    bias_layer3 = params[num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2 + num_weights_layer3 : num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2 + num_weights_layer3 + num_bias_layer3]
    weights_layer4 = params[num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2 + num_weights_layer3 + num_bias_layer3 : num_weights_layer1 + num_bias_layer1 + num_weights_layer2 + num_bias_layer2 + num_weights_layer3 + num_bias_layer3 + num_weights_layer4].reshape((num_layers_units[2], num_output_units))
    bias_layer4 = params[-num_bias_layer4:]

    model.layers[0].set_weights([weights_layer1, bias_layer1])
    model.layers[1].set_weights([weights_layer2, bias_layer2])
    model.layers[2].set_weights([weights_layer3, bias_layer3])
    model.layers[3].set_weights([weights_layer4, bias_layer4])

    # Compile the model with SGD optimizer and MSE loss
    model.compile(optimizer=SGD(learning_rate=0.01), loss='mean_squared_error')
    
    # Latih model
    print(f'Fit ke-{index}')
    model.fit(dataset, epochs=100, verbose=1)
    print(f'Fit ke-{index} End')
    
    print('Predict: ', model.predict([[-145.0, -145.0, -145.0, -145.0]]))

    # Evaluate the model using MSE
    y_pred = model.predict(X)
    mse = mean_squared_error(y, y_pred)
    print(f'MSE ke-{index}', mse)
    index += 1

    return mse


# Define the bounds for the parameters
lower_bound = [-1.0] * total_parameters  # Adjust this size as needed
upper_bound = [1.0] * total_parameters   # Adjust this size as needed

# Perform PSO optimization using pyswarm
best_params, best_cost = pso(
    fitness_function, lower_bound, upper_bound, swarmsize=10, maxiter=50)

num_weights_layer1: 80
num_bias_layer1: 20
num_weights_layer2: 600
num_bias_layer2: 30
num_weights_layer3: 600
num_bias_layer3: 20
num_weights_layer4: 20
num_bias_layer4: 1
4 [20, 30, 20] 1 80 1371
Proses ke-0
Params (20,) 80
Fit ke-0
Epoch 1/100


862/862 [==============================] - 1s 1ms/step - loss: 349315.9062
Epoch 2/100
862/862 [==============================] - 1s 1ms/step - loss: 349298.5625
Epoch 3/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.4062
Epoch 4/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.2812
Epoch 5/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.5000
Epoch 6/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.2812
Epoch 7/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.4688
Epoch 8/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.5625
Epoch 9/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.5625
Epoch 10/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.7500
Epoch 11/100
862/862 [==============================] - 1s 1ms/step - loss: 349211.6562
Epoch 12/100
862/862 [==============================]

KeyboardInterrupt: 

In [ ]:
# Create the ANN model with the specified architecture
best_model = keras.Sequential()
best_model.add(keras.layers.Dense(num_layers_units[0], input_dim=num_input_units, activation='sigmoid'))
best_model.add(keras.layers.Dense(num_layers_units[1], activation='sigmoid'))
best_model.add(keras.layers.Dense(num_layers_units[2], activation='sigmoid'))
best_model.add(keras.layers.Dense(num_output_units, activation='sigmoid'))

best_model.layers[0].set_weights([
    best_params[:num_weights_layer1].reshape((num_input_units, num_layers_units[0])),
    best_params[num_weights_layer1:num_weights_layer1 + num_bias_layer1]])
best_model.layers[1].set_weights([
    best_params[-num_weights_layer2 - num_bias_layer2:-num_bias_layer2].reshape((30, 20)),
    best_params[-num_bias_layer2:-num_bias_layer2 + num_bias_layer2]])
best_model.layers[2].set_weights([
    best_params[-num_weights_layer3 - num_bias_layer3:-num_bias_layer3].reshape((20, 30)),
    best_params[-num_bias_layer3:-num_bias_layer3 + num_bias_layer3]])
best_model.layers[3].set_weights([
    best_params[-num_weights_layer4 - num_bias_layer4:-num_bias_layer4].reshape((30, 1)),
    best_params[-num_bias_layer4:]])

In [ ]:
best_model.predict([[-145.0, -145.0, -145.0, -145.0]])

1/1 [==============================] - 0s 32ms/step


array([[-0.55892867]], dtype=float32)

In [ ]:
# Compile the model with SGD optimizer and MSE loss
best_model.compile(optimizer=SGD(learning_rate=0.01),
                   loss='mean_squared_error')


# Simpan model terbaik ke dalam berkas
best_model.save(f"ann_pso_v{VERSION}.keras")

In [ ]:
model = tf.keras.models.load_model(f"ann_pso_v{VERSION}.keras")

In [ ]:
model.predict([[-145.0, -145.0, -145.0, -145.0]])

1/1 [==============================] - 0s 29ms/step


array([[-0.55892867]], dtype=float32)